In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# Importação

In [2]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
im=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv').drop(columns='date')

# Elimina ano incompleto
im=im[im.CO_ANO!=2024]

im=im[im.KG_LIQUIDO>0]

# Modifica unidade de peso
im.KG_LIQUIDO=im.KG_LIQUIDO/10**6
im=im.rename(columns={'KG_LIQUIDO':'kton'})
im.head(1)

,CO_ANO,CO_MES,kton
0,1997,3,7.505776


# Chuvas

In [3]:
ch=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
ch.tail(1)

,CO_ANO,CO_MES,mm
264,2023,12,78.6


In [4]:
list=[6,7,8]

for ad in list:

    c=ch.copy()
    
    # Adianta os meses
    c.CO_MES=c.CO_MES+ad
    
    # Corriges meses fora da faixa
    for adi in range(1,ad+1):
        c.loc[c.CO_MES==12+adi,'CO_ANO']=c.loc[c.CO_MES==12+adi]['CO_ANO']+1
        c.loc[c.CO_MES==12+adi,'CO_MES']=adi
    
    c=c.rename(columns={'mm':'mm_'+str(ad)})

    if ad == list[0]:
        c1=c.copy()
    else:
        c1=c1.merge(c)
c1

,CO_ANO,CO_MES,mm_6,mm_7,mm_8
0,2002,8,0.0,0.0,23.2
1,2002,9,28.0,0.0,0.0
2,2002,10,17.8,28.0,0.0
3,2002,11,22.3,17.8,28.0
4,2002,12,2.1,22.3,17.8
...,...,...,...,...,...
258,2024,2,14.8,0.0,0.0
259,2024,3,0.8,14.8,0.0
260,2024,4,38.7,0.8,14.8
261,2024,5,39.8,38.7,0.8


Executa os testes

In [5]:
q=pd.DataFrame()
#q=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv')

from sklearn.ensemble import RandomForestRegressor as RFR
# Define hiperparametro
md=1
# Instância o regresssor
RFR=RFR(max_depth=md, random_state=1)

i=im.copy()

        

# Funde chuvas com importações
i=i.merge(c1)
i

,CO_ANO,CO_MES,kton,mm_6,mm_7,mm_8
0,2002,8,78.232067,0.0,0.0,23.2
1,2002,9,8.362661,28.0,0.0,0.0
2,2002,10,46.987128,17.8,28.0,0.0
3,2002,11,92.415166,22.3,17.8,28.0
4,2002,12,10.949629,2.1,22.3,17.8
...,...,...,...,...,...,...
251,2023,8,96.753794,19.9,202.1,239.3
252,2023,9,172.570477,83.2,19.9,202.1
253,2023,10,240.914358,50.5,83.2,19.9
254,2023,11,90.119779,11.6,50.5,83.2


In [6]:
i[i.kton==0]

,CO_ANO,CO_MES,kton,mm_6,mm_7,mm_8


In [7]:
i=i[i.kton!=0]

In [8]:
# Recria coluna de data
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1').astype(int)

# Descarta coluna Ano, substituida por data
i=i.drop(columns='CO_ANO')

# Formata mês como informação categórica
i.CO_MES=i.CO_MES.astype('category')

i=pd.get_dummies(i, prefix_sep='~')

for a in range(1,len(i)):


    for train_size_mes in [10*12]:


        train=i.iloc[:a,:].tail(train_size_mes)
        
        if len(train)>=train_size_mes:


            
            test=i.iloc[a:a+1,:]
                        
            # Train
            X_train_tratamento=train.drop(columns='kton')
            X_train_testemunha=train.drop(columns=['kton','mm_6','mm_7','mm_8'])
            y_train=train.kton
    
            # Test
            X_test_tratamento  =test.drop(columns='kton')
            X_test_testemunha=test.drop(columns=['kton','mm_6','mm_7','mm_8'])
            y_test=test.kton
                
            RFR.fit(X_train_tratamento,y_train)
            
            test_pred=test     .assign(kton_pred_tratamento  =RFR.predict(X_test_tratamento  ).astype(int))
            
            RFR.fit(X_train_testemunha,y_train)
            
            test_pred=test_pred.assign(kton_pred_testemunha=RFR.predict(X_test_testemunha).astype(int))

            test_pred=test_pred.assign(ad=ad,train_size_anos=train_size_mes/12,md=md)
            
            q=pd.concat([q,test_pred])

q['erro_tratamento']=np.abs(q.kton-q.kton_pred_tratamento)
q['erro_testemunha']=np.abs(q.kton-q.kton_pred_testemunha)

print('erro_tratamento.mean',round(q.erro_tratamento.mean(),4))
print('erro_testemunha.mean',round(q.erro_testemunha.mean(),4))

erro_tratamento.mean 38.713
erro_testemunha.mean 43.7664


Salva o resultado dos testes

In [9]:
q=pd.concat([q[[x for x in q.columns if x not in q.columns[5:17]]],pd.from_dummies(q[q.columns[5:17]],sep='~')],axis=1)

In [10]:
q.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q_mm.csv', index=False)

In [11]:
q

,kton,mm_6,mm_7,mm_8,date,kton_pred_tratamento,kton_pred_testemunha,ad,train_size_anos,md,erro_tratamento,erro_testemunha,CO_MES
120,44.226473,122.2,97.6,150.1,1346457600000000000,44,69,8,10.0,1,0.226473,24.773527,9
121,81.548957,22.4,122.2,97.6,1349049600000000000,58,54,8,10.0,1,23.548957,27.548957,10
122,93.418343,7.0,22.4,122.2,1351728000000000000,36,51,8,10.0,1,57.418343,42.418343,11
123,42.681645,0.0,7.0,22.4,1354320000000000000,34,52,8,10.0,1,8.681645,9.318355,12
124,15.768908,0.0,0.0,7.0,1356998400000000000,36,51,8,10.0,1,20.231092,35.231092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,96.753794,19.9,202.1,239.3,1690848000000000000,139,140,8,10.0,1,42.246206,43.246206,8
252,172.570477,83.2,19.9,202.1,1693526400000000000,129,136,8,10.0,1,43.570477,36.570477,9
253,240.914358,50.5,83.2,19.9,1696118400000000000,133,135,8,10.0,1,107.914358,105.914358,10
254,90.119779,11.6,50.5,83.2,1698796800000000000,143,145,8,10.0,1,52.880221,54.880221,11
